# **Skenario 3 - CNN MobileNet**

Skenario 3 menggunakan model Convolutional Neural Network (CNN) dengan arsitektur MobileNet untuk melakukan klasifikasi pada data citra otak.

---

## 1. Import packages

Import library yang digunakan untuk preprocessing data, membangun model, dan visualisas

In [7]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io

## 2. Data Preparation

### 2.1 Load dan Eksplorasi Data
cari dan cetak data `selected_image.npz` di dalam struktur direktori dan menampilkan informasi tentang jumlah slice untuk setiap label dan plane

In [2]:
directory_path = r"D:\Documents\Kuliah\.SKRIPSI\KLASIFIKASI ALZHEIMER\Coding\Notebook\Preprocessing\selected_image.npz"
loaded_data = np.load(directory_path, allow_pickle=True)

combined_slices = loaded_data['combined_slices'].item()

for label, plane_slices in combined_slices.items():
    for plane, result_slices in plane_slices.items():
        total_slices = len(result_slices)
        
        print(f"Label: {label}, Plane: {plane}, Jumlah Total Slice: {total_slices}")

        # Menampilkan bentuk (shape) dari setiap slice
        for i, (position, resized_slice) in enumerate(result_slices):
            print(f"  Resized Slice {i+1} - Position: {position}, Shape: {resized_slice.shape}")

Label: AD, Plane: coronal, Jumlah Total Slice: 60
  Resized Slice 1 - Position: 107, Shape: (224, 224)
  Resized Slice 2 - Position: 111, Shape: (224, 224)
  Resized Slice 3 - Position: 110, Shape: (224, 224)
  Resized Slice 4 - Position: 122, Shape: (224, 224)
  Resized Slice 5 - Position: 121, Shape: (224, 224)
  Resized Slice 6 - Position: 123, Shape: (224, 224)
  Resized Slice 7 - Position: 132, Shape: (224, 224)
  Resized Slice 8 - Position: 131, Shape: (224, 224)
  Resized Slice 9 - Position: 133, Shape: (224, 224)
  Resized Slice 10 - Position: 136, Shape: (224, 224)
  Resized Slice 11 - Position: 135, Shape: (224, 224)
  Resized Slice 12 - Position: 137, Shape: (224, 224)
  Resized Slice 13 - Position: 136, Shape: (224, 224)
  Resized Slice 14 - Position: 137, Shape: (224, 224)
  Resized Slice 15 - Position: 135, Shape: (224, 224)
  Resized Slice 16 - Position: 133, Shape: (224, 224)
  Resized Slice 17 - Position: 132, Shape: (224, 224)
  Resized Slice 18 - Position: 131, Shape

### 2.2 Pembagian Data

Menyiapkan data dengan menggabungkan slice dan melakukan label encoding.

In [3]:
X = []
y = []

for label, plane_slices in combined_slices.items():
    for plane, result_slices in plane_slices.items():
        X.extend([np.expand_dims(slice_data, axis=-1) for _, slice_data in result_slices])
        y.extend([label] * len(result_slices))

X = np.array(X)
y = np.array(y)

In [4]:
# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## 3. Data Preprocessing

### 3.1 Normalisasi dan Penyesuaian Format Citra
Mengatur parameter untuk cross-validation dan menentukan nilai mean dan std untuk normalisasi data.

In [5]:
from sklearn.model_selection import StratifiedKFold

# Perform K-Fold cross-validation
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

mean = 0.485
std = 0.229

### 3.2 Cross - Validation
Melakukan cross-validation dengan 5 fold. Di dalam setiap fold

In [6]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Inisialisasi k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

## 4. Training

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(X, y), start=1):
    # Model ResNet50 sbg base model
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))

    # Setel base model gabisa dilatih
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Menyiapkan data train dan test dg label encoding
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    y_train_encoded = to_categorical(y_train, num_classes=4)
    y_test_encoded = to_categorical(y_test, num_classes=4)

    # Normalisasi dan ubah gambar grayscale menjadi RGB
    X_train_normalized = (X_train / 255.0 - mean) / std
    X_test_normalized = (X_test / 255.0 - mean) / std

    X_train_rgb = np.repeat(X_train_normalized, 3, axis=-1)
    X_test_rgb = np.repeat(X_test_normalized, 3, axis=-1)

    # Latih model
    model.fit(X_train_rgb, y_train_encoded, epochs=10, validation_data=(X_test_rgb, y_test_encoded))

    # Evaluasi model
    _, accuracy_fold = model.evaluate(X_test_rgb, y_test_encoded)
    y_pred = model.predict(X_test_rgb)
    y_pred_labels = np.argmax(y_pred, axis=1)
    classification_rep_fold = classification_report(np.argmax(y_test_encoded, axis=1), y_pred_labels)

    # Simpan skor dan menampilkannya
    print(f"Fold {i} - Accuracy: {accuracy_fold}")
    print("Classification Report:")
    print(classification_rep_fold)

    cv_scores.append(accuracy_fold)